In [1]:
# Differentiable Augmentation for Data-Efficient GAN Training
# Shengyu Zhao, Zhijian Liu, Ji Lin, Jun-Yan Zhu, and Song Han
# https://arxiv.org/pdf/2006.10738

import torch
import torch.nn.functional as F
import random

def DiffAugment(x, policy='', channels_first=True):
    if policy:
        if not channels_first:
            x = x.permute(0, 3, 1, 2)
        for p in policy.split(','):
            for f in AUGMENT_FNS[p]:
                x = f(x)
        if not channels_first:
            x = x.permute(0, 2, 3, 1)
        x = x.contiguous()
    return x


def rand_brightness(x):
    x = x + (torch.rand(x.size(0), 1, 1, 1, dtype=x.dtype, device=x.device) - 0.5)
    return x


def rand_saturation(x):
    x_mean = x.mean(dim=1, keepdim=True)
    x = (x - x_mean) * (torch.rand(x.size(0), 1, 1, 1, dtype=x.dtype, device=x.device) * 2) + x_mean
    return x


def rand_contrast(x):
    x_mean = x.mean(dim=[1, 2, 3], keepdim=True)
    x = (x - x_mean) * (torch.rand(x.size(0), 1, 1, 1, dtype=x.dtype, device=x.device) + 0.5) + x_mean
    return x


def rand_translation(x, ratio=0.2):
    shift_x, shift_y = int(x.size(2) * ratio + 0.5), int(x.size(3) * ratio + 0.5)
    translation_x = torch.randint(-shift_x, shift_x + 1, size=[x.size(0), 1, 1], device=x.device)
    translation_y = torch.randint(-shift_y, shift_y + 1, size=[x.size(0), 1, 1], device=x.device)
    grid_batch, grid_x, grid_y = torch.meshgrid(
        torch.arange(x.size(0), dtype=torch.long, device=x.device),
        torch.arange(x.size(2), dtype=torch.long, device=x.device),
        torch.arange(x.size(3), dtype=torch.long, device=x.device),
    )
    grid_x = torch.clamp(grid_x + translation_x + 1, 0, x.size(2) + 1)
    grid_y = torch.clamp(grid_y + translation_y + 1, 0, x.size(3) + 1)
    x_pad = F.pad(x, [1, 1, 1, 1, 0, 0, 0, 0])
    x = x_pad.permute(0, 2, 3, 1).contiguous()[grid_batch, grid_x, grid_y].permute(0, 3, 1, 2)
    return x


def rand_cutout(x, ratio=0.5):
    if random.random() < 0.3:
        cutout_size = int(x.size(2) * ratio + 0.5), int(x.size(3) * ratio + 0.5)
        offset_x = torch.randint(0, x.size(2) + (1 - cutout_size[0] % 2), size=[x.size(0), 1, 1], device=x.device)
        offset_y = torch.randint(0, x.size(3) + (1 - cutout_size[1] % 2), size=[x.size(0), 1, 1], device=x.device)
        grid_batch, grid_x, grid_y = torch.meshgrid(
            torch.arange(x.size(0), dtype=torch.long, device=x.device),
            torch.arange(cutout_size[0], dtype=torch.long, device=x.device),
            torch.arange(cutout_size[1], dtype=torch.long, device=x.device),
        )
        grid_x = torch.clamp(grid_x + offset_x - cutout_size[0] // 2, min=0, max=x.size(2) - 1)
        grid_y = torch.clamp(grid_y + offset_y - cutout_size[1] // 2, min=0, max=x.size(3) - 1)
        mask = torch.ones(x.size(0), x.size(2), x.size(3), dtype=x.dtype, device=x.device)
        mask[grid_batch, grid_x, grid_y] = 0
        x = x * mask.unsqueeze(1)
    return x

def rand_rotate(x, ratio=0.5):
    k = random.randint(1,3)
    if random.random() < ratio:
        x = torch.rot90(x, k, [2,3])
    return x

AUGMENT_FNS = {
    'color': [rand_brightness, rand_saturation, rand_contrast],
    'translation': [rand_translation],
    'cutout': [rand_cutout],
    'rotate': [rand_rotate],
}

In [2]:
!pip install einops
!pip install timm

In [8]:
import torch
import torch.nn as nn

from einops import rearrange
from timm.models.layers import DropPath, to_2tuple, trunc_normal_


class MLP(nn.Module):
    def __init__(self, in_features, hidden_features=None, out_features=None, drop=0.):
        super(MLP, self).__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features

        self.net = nn.Sequential(
            nn.Linear(in_features, hidden_features),
            nn.GELU(),
            nn.Dropout(drop),
            nn.Linear(hidden_features, out_features),
            nn.Dropout(drop)
        )

    def forward(self, x):
        return self.net(x)


def window_partition(x, window_size):
    """
    Args:
        x: (B, H, W, C)
        window_size (int): window size
    Returns:
        windows: (num_windows*B, window_size, window_size, C)
    """
    B, H, W, C = x.shape
    x = x.view(B, H // window_size, window_size, W // window_size, window_size, C)
    windows = x.permute(0, 1, 3, 2, 4, 5).contiguous().view(-1, window_size, window_size, C)
    return windows


def window_reverse(windows, window_size, H, W):
    """
    Args:
        windows: (num_windows*B, window_size, window_size, C)
        window_size (int): Window size
        H (int): Height of image
        W (int): Width of image
    Returns:
        x: (B, H, W, C)
    """
    B = int(windows.shape[0] / (H * W / window_size / window_size))
    x = windows.view(B, H // window_size, W // window_size, window_size, window_size, -1)
    x = x.permute(0, 1, 3, 2, 4, 5).contiguous().view(B, H, W, -1)
    return x


class WindowAttention(nn.Module):
    r""" Window based multi-head self attention (W-MSA) module with relative position bias.
    It supports both of shifted and non-shifted window.
    Args:
        dim (int): Number of input channels.
        window_size (tuple[int]): The height and width of the window.
        num_heads (int): Number of attention heads.
        qkv_bias (bool, optional):  If True, add a learnable bias to query, key, value. Default: True
        qk_scale (float | None, optional): Override default qk scale of head_dim ** -0.5 if set
        attn_drop (float, optional): Dropout ratio of attention weight. Default: 0.0
        proj_drop (float, optional): Dropout ratio of output. Default: 0.0
    """

    def __init__(self, dim, window_size, num_heads, qkv_bias=True, qk_scale=None, attn_drop=0., proj_drop=0.):
        super().__init__()
        self.dim = dim
        self.window_size = window_size  # Wh, Ww
        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = qk_scale or head_dim ** -0.5

        # define a parameter table of relative position bias
        self.relative_position_bias_table = nn.Parameter(
            torch.zeros((2 * window_size[0] - 1) * (2 * window_size[1] - 1), num_heads))  # 2*Wh-1 * 2*Ww-1, nH

        # get pair-wise relative position index for each token inside the window
        coords_h = torch.arange(self.window_size[0])
        coords_w = torch.arange(self.window_size[1])
        coords = torch.stack(torch.meshgrid([coords_h, coords_w]))  # 2, Wh, Ww
        coords_flatten = torch.flatten(coords, 1)  # 2, Wh*Ww
        relative_coords = coords_flatten[:, :, None] - coords_flatten[:, None, :]  # 2, Wh*Ww, Wh*Ww
        relative_coords = relative_coords.permute(1, 2, 0).contiguous()  # Wh*Ww, Wh*Ww, 2
        relative_coords[:, :, 0] += self.window_size[0] - 1  # shift to start from 0
        relative_coords[:, :, 1] += self.window_size[1] - 1
        relative_coords[:, :, 0] *= 2 * self.window_size[1] - 1
        relative_position_index = relative_coords.sum(-1)  # Wh*Ww, Wh*Ww
        self.register_buffer("relative_position_index", relative_position_index)

        self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

        trunc_normal_(self.relative_position_bias_table, std=.02)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x, mask=None):
        """
        Args:
            x: input features with shape of (num_windows*B, N, C)
            mask: (0/-inf) mask with shape of (num_windows, Wh*Ww, Wh*Ww) or None
        """
        B_, N, C = x.shape
        qkv = self.qkv(x).reshape(B_, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
        q, k, v = qkv[0], qkv[1], qkv[2]  # make torchscript happy (cannot use tensor as tuple)

        q = q * self.scale
        attn = (q @ k.transpose(-2, -1))

        relative_position_bias = self.relative_position_bias_table[self.relative_position_index.view(-1)].view(
            self.window_size[0] * self.window_size[1], self.window_size[0] * self.window_size[1], -1)  # Wh*Ww,Wh*Ww,nH
        relative_position_bias = relative_position_bias.permute(2, 0, 1).contiguous()  # nH, Wh*Ww, Wh*Ww
        attn = attn + relative_position_bias.unsqueeze(0)

        if mask is not None:
            nW = mask.shape[0]
            attn = attn.view(B_ // nW, nW, self.num_heads, N, N) + mask.unsqueeze(1).unsqueeze(0)
            attn = attn.view(-1, self.num_heads, N, N)
            attn = self.softmax(attn)
        else:
            attn = self.softmax(attn)

        attn = self.attn_drop(attn)

        x = (attn @ v).transpose(1, 2).reshape(B_, N, C)
        x = self.proj(x)
        x = self.proj_drop(x)
        return x


class SwinTransformerBlock(nn.Module):
    r""" Swin Transformer Block.
    Args:
        dim (int): Number of input channels.
        input_resolution (tuple[int]): Input resulotion.
        num_heads (int): Number of attention heads.
        window_size (int): Window size.
        shift_size (int): Shift size for SW-MSA.
        mlp_ratio (float): Ratio of mlp hidden dim to embedding dim.
        qkv_bias (bool, optional): If True, add a learnable bias to query, key, value. Default: True
        qk_scale (float | None, optional): Override default qk scale of head_dim ** -0.5 if set.
        drop (float, optional): Dropout rate. Default: 0.0
        attn_drop (float, optional): Attention dropout rate. Default: 0.0
        drop_path (float, optional): Stochastic depth rate. Default: 0.0
        act_layer (nn.Module, optional): Activation layer. Default: nn.GELU
        norm_layer (nn.Module, optional): Normalization layer.  Default: nn.LayerNorm
    """

    def __init__(self, dim, input_resolution, num_heads, window_size=2, shift_size=0,
                 mlp_ratio=4., qkv_bias=True, qk_scale=None, drop=0., attn_drop=0., drop_path=0.,
                 act_layer=nn.GELU, norm_layer=nn.LayerNorm):
        super().__init__()
        self.dim = dim
        self.input_resolution = input_resolution
        self.num_heads = num_heads
        self.window_size = window_size
        self.shift_size = shift_size
        self.mlp_ratio = mlp_ratio
        if min(self.input_resolution) <= self.window_size:
            # if window size is larger than input resolution, we don't partition windows
            self.shift_size = 0
            self.window_size = min(self.input_resolution)
        assert 0 <= self.shift_size < self.window_size, "shift_size must in 0-window_size"

        self.norm1 = norm_layer(dim)
        self.attn = WindowAttention(
            dim, window_size=to_2tuple(self.window_size), num_heads=num_heads,
            qkv_bias=qkv_bias, qk_scale=qk_scale, attn_drop=attn_drop, proj_drop=drop)

        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()
        self.norm2 = norm_layer(dim)
        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp = MLP(in_features=dim, hidden_features=mlp_hidden_dim, drop=drop)

        if self.shift_size > 0:
            # calculate attention mask for SW-MSA
            H, W = self.input_resolution
            img_mask = torch.zeros((1, H, W, 1))  # 1 H W 1
            h_slices = (slice(0, -self.window_size),
                        slice(-self.window_size, -self.shift_size),
                        slice(-self.shift_size, None))
            w_slices = (slice(0, -self.window_size),
                        slice(-self.window_size, -self.shift_size),
                        slice(-self.shift_size, None))
            cnt = 0
            for h in h_slices:
                for w in w_slices:
                    img_mask[:, h, w, :] = cnt
                    cnt += 1

            mask_windows = window_partition(img_mask, self.window_size)  # nW, window_size, window_size, 1
            mask_windows = mask_windows.view(-1, self.window_size * self.window_size)
            attn_mask = mask_windows.unsqueeze(1) - mask_windows.unsqueeze(2)
            attn_mask = attn_mask.masked_fill(attn_mask != 0, float(-100.0)).masked_fill(attn_mask == 0, float(0.0))
        else:
            attn_mask = None

        self.register_buffer("attn_mask", attn_mask)

    def forward(self, x):
        H, W = self.input_resolution
        B, L, C = x.shape
        assert L == H * W, "input feature has wrong size"

        shortcut = x
        x = self.norm1(x)
        x = x.view(B, H, W, C)

        # cyclic shift
        if self.shift_size > 0:
            shifted_x = torch.roll(x, shifts=(-self.shift_size, -self.shift_size), dims=(1, 2))
        else:
            shifted_x = x

        # partition windows
        x_windows = window_partition(shifted_x, self.window_size)  # nW*B, window_size, window_size, C
        x_windows = x_windows.view(-1, self.window_size * self.window_size, C)  # nW*B, window_size*window_size, C

        # W-MSA/SW-MSA
        attn_windows = self.attn(x_windows, mask=self.attn_mask)  # nW*B, window_size*window_size, C

        # merge windows
        attn_windows = attn_windows.view(-1, self.window_size, self.window_size, C)
        shifted_x = window_reverse(attn_windows, self.window_size, H, W)  # B H' W' C

        # reverse cyclic shift
        if self.shift_size > 0:
            x = torch.roll(shifted_x, shifts=(self.shift_size, self.shift_size), dims=(1, 2))
        else:
            x = shifted_x
        x = x.view(B, H * W, C)

        # FFN
        x = shortcut + self.drop_path(x)
        x = x + self.drop_path(self.mlp(self.norm2(x)))

        return x


class PatchMerging(nn.Module):
    r""" Patch Merging Layer.
    Args:
        input_resolution (tuple[int]): Resolution of input feature.
        dim (int): Number of input channels.
        norm_layer (nn.Module, optional): Normalization layer.  Default: nn.LayerNorm
    """

    def __init__(self, input_resolution, dim, norm_layer=nn.LayerNorm):
        super().__init__()
        self.input_resolution = input_resolution
        self.dim = dim
        self.reduction = nn.Linear(4 * dim, 2 * dim, bias=False)
        self.norm = norm_layer(4 * dim)

    def forward(self, x):
        """
        x: B, H*W, C
        """
        H, W = self.input_resolution
        B, L, C = x.shape
        assert L == H * W, "input feature has wrong size"
        assert H % 2 == 0 and W % 2 == 0, f"x size ({H}*{W}) are not even."

        x = x.view(B, H, W, C)

        x0 = x[:, 0::2, 0::2, :]  # B H/2 W/2 C
        x1 = x[:, 1::2, 0::2, :]  # B H/2 W/2 C
        x2 = x[:, 0::2, 1::2, :]  # B H/2 W/2 C
        x3 = x[:, 1::2, 1::2, :]  # B H/2 W/2 C
        x = torch.cat([x0, x1, x2, x3], -1)  # B H/2 W/2 4*C
        x = x.view(B, -1, 4 * C)  # B H/2*W/2 4*C

        x = self.norm(x)
        x = self.reduction(x)

        return x


class PatchExpand(nn.Module):
    def __init__(self, input_resolution, dim, dim_scale=2, norm_layer=nn.LayerNorm):
        super().__init__()
        self.input_resolution = input_resolution
        self.dim = dim
        self.expand = nn.Linear(dim, 2*dim, bias=False) if dim_scale==2 else nn.Identity()
        self.norm = norm_layer(dim // dim_scale)

    def forward(self, x):
        """
        x: B, H*W, C
        """
        H, W = self.input_resolution
        x = self.expand(x)
        B, L, C = x.shape
        assert L == H * W, "input feature has wrong size"

        x = x.view(B, H, W, C)
        x = rearrange(x, 'b h w (p1 p2 c) -> b (h p1) (w p2) c', p1=2, p2=2, c=C//4)
        x = x.view(B, -1, C // 4)
        x = self.norm(x)

        return x


class PatchEmbed(nn.Module):
    r""" Image to Patch Embedding
    Args:
        img_size (int): Image size.  Default: 224.
        patch_size (int): Patch token size. Default: 4.
        in_chans (int): Number of input image channels. Default: 3.
        embed_dim (int): Number of linear projection output channels. Default: 96.
        norm_layer (nn.Module, optional): Normalization layer. Default: None
    """

    def __init__(self, img_size=32, patch_size=4, in_chans=3, embed_dim=96, norm_layer=nn.LayerNorm):
        super().__init__()
        img_size = to_2tuple(img_size)
        patch_size = to_2tuple(patch_size)
        patches_resolution = [img_size[0] // patch_size[0], img_size[1] // patch_size[1]]
        self.img_size = img_size
        self.patch_size = patch_size
        self.patches_resolution = patches_resolution
        self.num_patches = patches_resolution[0] * patches_resolution[1]

        self.in_chans = in_chans
        self.embed_dim = embed_dim

        self.proj = nn.Conv2d(in_chans, embed_dim, kernel_size=patch_size, stride=patch_size)
        if norm_layer is not None:
            self.norm = norm_layer(embed_dim)
        else:
            self.norm = None

    def forward(self, x):
        B, C, H, W = x.shape
        # fixme look at relaxing size constraints
        assert H == self.img_size[0] and W == self.img_size[1], \
            f"Input image size ({H}*{W}) doesn't match model ({self.img_size[0]}*{self.img_size[1]})."
        x = self.proj(x).flatten(2).transpose(1, 2)  # B Ph*Pw C
        if self.norm is not None:
            x = self.norm(x)

        return x


class TransformerEncoder(nn.Module):
    """ A basic Swin Transformer layer for one stage.
    Args:
        dim (int): Number of input channels.
        input_resolution (tuple[int]): Input resolution.
        depth (int): Number of blocks.
        num_heads (int): Number of attention heads.
        window_size (int): Local window size.
        mlp_ratio (float): Ratio of mlp hidden dim to embedding dim.
        qkv_bias (bool, optional): If True, add a learnable bias to query, key, value. Default: True
        qk_scale (float | None, optional): Override default qk scale of head_dim ** -0.5 if set.
        drop (float, optional): Dropout rate. Default: 0.0
        attn_drop (float, optional): Attention dropout rate. Default: 0.0
        norm_layer (nn.Module, optional): Normalization layer. Default: nn.LayerNorm
        downsample (nn.Module | None, optional): Downsample layer at the end of the layer. Default: None
    """

    def __init__(self, dim, input_resolution, depth, num_heads=4, window_size=2,
                 mlp_ratio=4., qkv_bias=True, qk_scale=None, drop=0., attn_drop=0.,
                 drop_path=0., norm_layer=nn.LayerNorm):
        super().__init__()
        self.dim = dim
        self.input_resolution = input_resolution
        self.depth = depth

        # build blocks
        self.blocks = nn.ModuleList([
            SwinTransformerBlock(dim=dim, input_resolution=input_resolution,
                                 num_heads=num_heads, window_size=window_size,
                                 shift_size=0 if (i % 2 == 0) else window_size // 2,
                                 mlp_ratio=mlp_ratio,
                                 qkv_bias=qkv_bias, qk_scale=qk_scale,
                                 drop=drop, attn_drop=attn_drop,
                                 drop_path=drop_path[i] if isinstance(drop_path, list) else drop_path,
                                 norm_layer=norm_layer)
            for i in range(depth)])

    def forward(self, x):
        for block in self.blocks:
            x = block(x)
        return x



class Generator(nn.Module):
    def __init__(self, depth=[4, 3, 2], initial_size=8, dim=96, heads=4, mlp_ratio=4, window_size=2):
        super(Generator, self).__init__()
        self.initial_size = initial_size
        self.dim = dim
        self.depth = depth
        self.heads = heads
        self.mlp_ratio = mlp_ratio
        self.window_size = window_size

        # N, 1024 -> N, 8 * 8 * 96
        self.mlp = nn.Linear(128, (self.initial_size ** 2) * self.dim)

        # N, 8 * 8, 96 -> N, 8 * 8, 96
        self.decode1 = TransformerEncoder(dim=dim, input_resolution=[self.initial_size, self.initial_size], depth=self.depth[0], \
            num_heads=self.heads, window_size=self.window_size, mlp_ratio=mlp_ratio)
        # N, 8 * 8, 96 -> N, 16 * 16, 48
        self.up1 = PatchExpand([self.initial_size, self.initial_size], dim=dim)

        # N, 16 * 16, 48 -> N, 16 * 16, 48
        self.decode2 = TransformerEncoder(dim=(dim // 2), input_resolution=[self.initial_size * 2, self.initial_size * 2], depth=self.depth[1], \
            num_heads=self.heads, window_size=2, mlp_ratio=mlp_ratio)
        # N, 16 * 16, 48 -> N, 32 * 32, 24
        self.up2 = PatchExpand([self.initial_size * 2, self.initial_size * 2], dim=(dim // 2))

        # N, 32 * 32, 24 -> N, 32 * 32, 24
        self.decode3 = TransformerEncoder(dim=(dim // 4), input_resolution=[self.initial_size * 4, self.initial_size * 4], depth=self.depth[2], \
            num_heads=self.heads, window_size=2, mlp_ratio=mlp_ratio)

        # N, 24, 32, 32 -> N, 3, 32, 32
        self.linear = nn.Sequential(nn.Conv2d(self.dim // 4, 3, 1, 1, 0))

        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)

    def forward(self, noise):
        # N, 8 * 8 * 384 -> N, 8 * 8, 96
        x = self.mlp(noise).view(-1, self.initial_size ** 2, self.dim)

        # N, 8 * 8, 96 -> N, 8 * 8, 96
        x = self.decode1(x)
        # N, 8 * 8, 96 -> N, 16 * 16, 48
        x = self.up1(x)
        # N, 16 * 16, 48 -> N, 16 * 16, 48
        x = self.decode2(x)
        # N, 16 * 16, 48 -> N, 32 * 32, 24
        x = self.up2(x)
        # N, 32 * 32, 24 -> N, 32 * 32, 24
        x = self.decode3(x)
        # N, 32 * 32, 24 -> N, 24, 32 * 32 -> N, 24, 32, 32 -> N, 3, 32, 32
        x = self.linear(x.permute(0, 2, 1).view(-1, self.dim // 4, self.initial_size * 4, self.initial_size * 4))

        return x


class Discriminator(nn.Module):
    def __init__(self, diff_aug, image_size=32, patch_size=4, input_channels=3, dim=96, depth=[2, 2, 2], heads=4, \
        mlp_ratio=4, drop_rate=0., window_size=2, num_classes=1):
        super(Discriminator, self).__init__()

        if image_size % patch_size != 0:
            raise ValueError('Image size must be divisible by patch size.')
        # num_patches: 8 * 8
        num_patches = (image_size // patch_size) ** 2
        self.diff_aug = diff_aug
        self.patch_size = patch_size
        self.depth = depth
        self.dim = dim
        self.heads = heads
        self.window_size = window_size

        # split image into non-overlapping patches
        # N, 3, 32, 32 -> N, 8 * 8, 96
        self.patch_embed = PatchEmbed(img_size=image_size, patch_size=patch_size, in_chans=input_channels, embed_dim=dim)

        # absolute position embedding
        # N, 64, 96
        self.positional_embedding = nn.Parameter(torch.zeros(1, num_patches, dim))
        nn.init.trunc_normal_(self.positional_embedding, std=0.02)
        self.droprate = nn.Dropout(p=drop_rate)

        # N, 8 * 8, 96 -> N, 8 * 8, 96
        self.encode1 = TransformerEncoder(dim=self.dim, input_resolution=[image_size // 4, image_size // 4], depth=self.depth[0], \
            num_heads=self.heads, window_size=self.window_size, mlp_ratio=mlp_ratio)
        # N, 8 * 8, 96 -> N, 4 * 4, 192
        self.down1 = PatchMerging(input_resolution=[image_size // 4, image_size // 4], dim=self.dim)

        # N, 4 * 4, 192 -> N, 4 * 4, 192
        self.encode2 = TransformerEncoder(dim=(self.dim * 2), input_resolution=[image_size // 8, image_size // 8], depth=self.depth[1], \
            num_heads=self.heads, window_size=self.window_size, mlp_ratio=mlp_ratio)
        # N, 4 * 4, 192 -> N, 2 * 2, 384
        self.down2 = PatchMerging(input_resolution=[image_size // 8, image_size // 8], dim=(self.dim * 2))

        # N, 2 * 2, 384 -> N, 2 * 2, 384
        self.encode3 = TransformerEncoder(dim=(self.dim * 4), input_resolution=[image_size // 16, image_size // 16], depth=self.depth[2], \
            num_heads=self.heads, window_size=self.window_size, mlp_ratio=mlp_ratio)

        # N, 2 * 2, 384 -> N, 1
        self.norm = nn.LayerNorm(dim * 4)
        self.out = nn.Linear(dim * 4, num_classes)

        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            nn.init.trunc_normal_(m.weight, std=.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)

    def forward(self, x):
        # data augmentation 
        x = DiffAugment(x, self.diff_aug)

        # N, 64, 96
        x = self.patch_embed(x)
        x += self.positional_embedding
        x = self.droprate(x)

        # N, 8 * 8, 96 -> N, 8 * 8, 96
        x = self.encode1(x)
        # N, 8 * 8, 96 -> N, 4 * 4, 192
        x = self.down1(x)
        # N, 4 * 4, 192 -> N, 4 * 4, 192
        x = self.encode2(x)
        # N, 4 * 4, 192 -> N, 2 * 2, 384
        x = self.down2(x)
        # N, 2 * 2, 384 -> N, 2 * 2, 384
        x = self.encode3(x)
        # N, 2 * 2, 384 -> N, 2 * 2, 384
        x = self.norm(x)
        # N, 2 * 2, 384 -> N, 1
        x = self.out(x[:, 0])
        
        return x

In [9]:
import torch
import random

import numpy as np
import torch.backends.cudnn as cudnn


def noise(imgs, latent_dim):
    return torch.FloatTensor(np.random.normal(0, 1, (imgs.shape[0], latent_dim)))

def gener_noise(gener_batch_size, latent_dim):
    return torch.FloatTensor(np.random.normal(0, 1, (gener_batch_size, latent_dim)))

def calculate_gradient_penalty(model, real_images, fake_images, constant=1.0, lamb=10.0):
    """Calculates the gradient penalty loss for WGAN GP"""
    # Random weight term for interpolation between real and fake data
    alpha = torch.Tensor(np.random.random((real_images.size(0), 1, 1, 1))).to(real_images.get_device())
    # Get random interpolation between real and fake data
    interpolates = (alpha * real_images + ((1 - alpha) * fake_images)).requires_grad_(True)

    model_interpolates = model(interpolates)
    grad_outputs = torch.ones([real_images.shape[0], 1], requires_grad=False).to(real_images.get_device())

    # Get gradient w.r.t. interpolates
    gradients = torch.autograd.grad(
        outputs=model_interpolates,
        inputs=interpolates,
        grad_outputs=grad_outputs,
        create_graph=True,
        retain_graph=True,
        only_inputs=True,
    )[0]
    gradients = gradients.contiguous().view(gradients.size(0), -1)
    gradient_penalty = (((gradients + 1e-16).norm(2, dim=1) - constant) ** 2).mean() * lamb
    
    return gradient_penalty

# wgan_loss
def wgan_loss(pred, real_or_not=True):
    if real_or_not:
        return - torch.mean(pred)
    else:
        return torch.mean(pred)


# Source from "https://github.com/ultralytics/yolov5/blob/master/utils/torch_utils.py"
def init_torch_seeds(seed: int = 0):
    r""" Sets the seed for generating random numbers. Returns a
    Args:
        seed (int): The desired seed.
    """
    # Speed-reproducibility tradeoff https://pytorch.org/docs/stable/notes/randomness.html
    if seed == 0:  # slower, more reproducible
        cudnn.deterministic = True
        cudnn.benchmark = False
    else:  # faster, less reproducible
        cudnn.deterministic = False
        cudnn.benchmark = True

    print("Initialize random seed.")
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)


In [5]:
from torchvision import transforms
from torch.utils.data import Dataset

import torch
from PIL import Image
import os

class Dataset(Dataset):
    def __init__(self, root, transform, mode='train'):
        super(Dataset, self).__init__()

        self.root = root
        self.transform = transform
        self.mode = mode

        data_dir = os.path.join(root, mode)
        self.file_list = os.listdir(data_dir)

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, index):
        img_path = os.path.join(self.root, self.mode, self.file_list[index])
        img = Image.open(img_path)
        img_out = self.transform(img)

        return img_out


def data_loader(root, batch_size=20, shuffle=True, img_size=32, mode='train'):    
    transform = transforms.Compose([
        transforms.Resize(size=(img_size, img_size)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])
    
    dset = Dataset(root, transform, mode=mode)
    
    if batch_size == 'all':
        batch_size = len(dset)
        
    dloader = torch.utils.data.DataLoader(dset,
                                          batch_size=batch_size,
                                          shuffle=shuffle,
                                          num_workers=0,
                                          drop_last=True)
    dlen = len(dset)
    
    return dloader, dlen

In [10]:
import torch
import torch.optim as optim
from torchvision.utils import make_grid, save_image

from tensorboardX import SummaryWriter
import time
import datetime
from tqdm import tqdm

# for reproductionary
init_torch_seeds(seed=12345)

class Solver():
    def __init__(self, root='dataset/anime_faces', result_dir='result', img_size=32, weight_dir='weight', load_weight=False,
                 batch_size=32, gener_batch_size=25, epochs=200, save_every=100, latent_dim=1024, n_critic=5, diff_aug=None, 
                 g_lr=0.0002, d_lr=0.0001, beta_1=0.5, beta_2=0.999, logdir=None):
        
        # cpu or gpu
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # load generator and discriminator
        self.G = Generator()
        self.G.to(self.device)
        self.D = Discriminator(diff_aug=diff_aug)
        self.D.to(self.device)

        # load training dataset
        self.train_loader, _ = data_loader(root=root, batch_size=batch_size, shuffle=True, 
                                                img_size=img_size, mode='train')

        # optimizer
        self.optim_D = optim.Adam(self.D.parameters(), lr=d_lr, betas=(beta_1, beta_2))
        self.optim_G = optim.Adam(self.G.parameters(), lr=g_lr, betas=(beta_1, beta_2))

        # Some hyperparameters
        self.latent_dim = latent_dim
        self.writer = SummaryWriter(logdir)

        # Extra things
        self.result_dir = result_dir
        self.weight_dir = weight_dir
        self.load_weight = load_weight
        self.epochs = epochs
        self.gener_batch_size = gener_batch_size
        self.img_size = img_size
        self.start_epoch = 0
        self.num_epoch = epochs
        self.save_every = save_every
        self.n_critic = n_critic
        
    '''
    <show_model >
    Print model architectures
    '''
    def show_model(self):
        print('================================ Discriminator for image =====================================')
        print(self.D)
        print('==========================================================================================\n\n')
        print('================================= Generator ==================================================')
        print(self.G)
        print('==========================================================================================\n\n')
        
    '''
        < set_train_phase >
        Set training phase
    '''
    def set_train_phase(self):
        self.D.train()
        self.G.train()
    
    '''
        < load_checkpoint >
        If you want to continue to train, load pretrained weight from checkpoint
    '''
    def load_checkpoint(self, checkpoint):
        print('Load model')
        self.D.load_state_dict(checkpoint['discriminator_image_state_dict'])
        self.G.load_state_dict(checkpoint['generator_state_dict'])
        self.optim_D.load_state_dict(checkpoint['optim_d'])
        self.optim_G.load_state_dict(checkpoint['optim_g'])
        self.start_epoch = checkpoint['epoch']
        
    '''
        < save_checkpoint >
        Save checkpoint
    '''
    def save_checkpoint(self, state, file_name):
        print('saving check_point')
        torch.save(state, file_name)
    
    '''
        < all_zero_grad >
        Set all optimizers' grad to zero 
    '''
    def all_zero_grad(self):
        self.optim_D.zero_grad()
        self.optim_G.zero_grad()

    '''
        < train >
        Train the D_image, D_latnet, G and E 
    '''
    def train(self):
        if self.load_weight is True:
            weight_name = 'checkpoint_{epoch}_epoch.pkl'.format(epoch=self.epochs)
            checkpoint = torch.load(os.path.join(self.weight_dir, weight_name))
            self.load_checkpoint(checkpoint)
        
        self.set_train_phase()
        self.show_model()

        print('====================     Training    Start... =====================')
        for epoch in range(self.start_epoch, self.num_epoch):
            start_time = time.time()

            for iters, img in tqdm(enumerate(self.train_loader)):
                # load real images
                real_imgs = img.type(torch.cuda.FloatTensor)

                # generate fake images
                noise = torch.cuda.FloatTensor(np.random.normal(0, 1, (img.shape[0], self.latent_dim)))
                fake_imgs = self.G(noise)

                ''' ----------------------------- 1. Train D ----------------------------- '''
                # discriminator predict 
                fake_validity = self.D(fake_imgs.detach())
                real_validity = self.D(real_imgs)

                # gradient_penalty
                gradient_penalty = calculate_gradient_penalty(self.D, real_imgs, fake_imgs.detach())
                # loss measures generator's ability to fool the discriminator
                err_d = wgan_loss(real_validity, real_or_not=True) + wgan_loss(fake_validity, real_or_not=False) 

                d_loss = gradient_penalty + err_d
                self.writer.add_scalars('losses', {'d_loss': d_loss, 'grad_penalty': gradient_penalty}, iters)

                # update D
                self.all_zero_grad()
                d_loss.backward()
                self.optim_D.step()

                ''' ----------------------------- 2. Train G ----------------------------- '''
                # train the generator every n_critic iterations
                if iters % self.n_critic == 0:
                    noise = torch.cuda.FloatTensor(np.random.normal(0, 1, (img.shape[0], self.latent_dim)))
                    generated_imgs = self.G(noise)
                    
                    fake_validity = self.D(generated_imgs)
                    g_loss = wgan_loss(fake_validity, real_or_not=True).to(self.device)

                    self.writer.add_scalars('losses', {'g_loss': g_loss}, iters)

                    # update G
                    self.all_zero_grad()
                    g_loss.backward()
                    self.optim_G.step()

                log_file = open('log.txt', 'w')
                log_file.write(str(epoch))

                # Print error, save intermediate result image and weight
                if epoch and iters % self.save_every == 0:
                    et = time.time() - start_time
                    et = str(datetime.timedelta(seconds=et))[:-7]
                    print('[Elapsed : %s /Epoch : %d / Iters : %d] => loss_d : %f / loss_g : %f / gradient_penalty : %f '\
                          %(et, epoch, iters, d_loss.item(), g_loss.item(), gradient_penalty.item()))

                    # Save intermediate result image
                    if os.path.exists(self.result_dir) is False:
                        os.makedirs(self.result_dir)

                    self.G.eval()
                    with torch.no_grad():
                        gene_noise = torch.cuda.FloatTensor(np.random.normal(0, 1, (self.gener_batch_size, self.latent_dim)))
                        generated_imgs= self.G(gene_noise)
                        sample_imgs = generated_imgs[:25]

                    img_name = 'generated_img_{epoch}_{iters}.jpg'.format(epoch=epoch, iters=(iters % len(self.train_loader)))
                    img_path = os.path.join(self.result_dir, img_name)

                    img_grid = make_grid(sample_imgs, nrow=5, normalize=True, scale_each=True)
                    save_image(img_grid, img_path, nrow=5, normalize=True, scale_each=True)  

                    # save intermediate weight
                    if os.path.exists(self.weight_dir) is False:
                        os.makedirs(self.weight_dir)  

            # Save weight at the end of every epoch
            if (epoch % 25) == 0:
                # self.save_weight(epoch=epoch)
                checkpoint = {
                    "generator_state_dict": self.G.state_dict(),
                    "discriminator_image_state_dict": self.D.state_dict(),
                    "optim_g": self.optim_G.state_dict(),
                    "optim_d": self.optim_D.state_dict(),
                    "epoch": epoch
                    }
                path_checkpoint = os.path.join(self.weight_dir, "checkpoint_{}_epoch.pkl".format(epoch))
                self.save_checkpoint(checkpoint, path_checkpoint)        

In [ ]:
import argparse
import os

def main(args):
    solver = Solver(root = args.root,
                    result_dir = args.result_dir,
                    img_size = args.img_size,
                    weight_dir = args.weight_dir,
                    batch_size = args.batch_size,
                    gener_batch_size = args.gener_batch_size,
                    g_lr = args.g_lr,
                    d_lr = args.d_lr,
                    beta_1 = args.beta_1,
                    beta_2 = args.beta_2,
                    save_every = args.save_every,
                    latent_dim = args.latent_dim,
                    n_critic = args.n_critic,
                    epochs = args.epochs,
                    diff_aug = args.diff_aug,
                    load_weight = args.load_weight,
                    )
                    
    solver.train()

if __name__ == '__main__':
    parser = argparse.ArgumentParser()

    parser.add_argument('--root', type=str, default='../input/anime64', help='Data location')
    parser.add_argument('--result_dir', type=str, default='./', help='Result images location')
    parser.add_argument('--img_size', type=int, default=32, help='Size of image for discriminator input.')
    parser.add_argument('--weight_dir', type=str, default='./', help='Weight location')
    parser.add_argument('--batch_size', type=int, default=50, help='Training batch size')
    parser.add_argument('--gener_batch_size', type=int, default=25, help='Batch size for generator.')
    parser.add_argument('--g_lr', type=float, default=0.0001, help='Learning rate')
    parser.add_argument('--d_lr', type=float, default=0.0002, help='Discriminator Learning rate')
    parser.add_argument('--beta_1', type=float, default=0.0, help='Beta1 for Adam')
    parser.add_argument('--beta_2', type=float, default=0.99, help='Beta2 for Adam')
    parser.add_argument('--save_every', type=int, default=200, help='How often do you want to see the result?')
    parser.add_argument('--latent_dim', type=int, default=128, help='Latent dimension.')
    parser.add_argument('--n_critic', type=int, default=5, help='n_critic.')
    parser.add_argument('--epochs', type=int, default=1000, help='Number of epoch.')
    parser.add_argument('--diff_aug', type=str, default="translation,cutout,color", help='Data Augmentation')
    parser.add_argument('--load_weight', type=bool, default=False, help='Load weight or not')
                        
    args = parser.parse_args([])
    main(args=args)